# **MLP** (Multi-Layer Perceptron)
<img src="docs/images/image-5.png" alt="mlp" width="550" />

In [1]:
import numpy as np
import math as m
import pandas as pd

## **ARCHITECTURE**

* **Entry: 8** attributtes
* **Hidden Layers (H): 1**
* **Neurons** layer **H: 3**
* **Neurons** layer **Out (O) = 2**

<div style="text-align:center;">
  <img src="docs/images/x1.png" alt="perceptron" width="500" />
</div>


In [2]:
entry_len = 8
H_len = 3
O_len = 2

---

## **OPTIMAL MATRIX LAYOUT**

### 1️⃣ **Input**

Each input datum (**one instance**):

$$
x = \begin{bmatrix} x_1 & x_2 & x_3 & x_4 \end{bmatrix}_{(1,4)}
$$

If you use a batch of size $N$:

$$
X \in \mathbb{R}^{N \times 4}
$$

---

### 2️⃣ **Hidden layer**

#### · **Weights**

$$
W^{(1)} \in \mathbb{R}^{3 \times 4}
$$

#### · **Bias**

$$
b^{(1)} \in \mathbb{R}^{1 \times 3}
$$

#### · **Pre‑activation computation (z)**

$$
Z^{(1)} = X \cdot W^{(1)^T} + b^{(1)}
$$

$$
Z^{(1)} \in \mathbb{R}^{N \times 3}
$$

#### · **Output computation (activation)**

$$
H^{(1)} = f(Z^{(1)})
$$

$$
H^{(1)} \in \mathbb{R}^{N \times 3}
$$

Here $f$ could be, for example, ReLU or **Sigmoid**, depending on the architecture. 

---

### **3️⃣ Output layer**

#### · **Weights**

$$
W^{(2)} \in \mathbb{R}^{2 \times 3}
$$

#### · **Bias**

$$
b^{(2)} \in \mathbb{R}^{1 \times 2}
$$

#### · **Pre‑activation computation (z)**

$$
Z^{(2)} = H^{(1)} \cdot W^{(2)^T} + b^{(2)}
$$

$$
Z^{(2)} \in \mathbb{R}^{N \times 2}
$$

#### · **Final output computation (activation)**

$$
Y = g(Z^{(2)})
$$

$$
Y \in \mathbb{R}^{N \times 2}
$$

Here $g$ could be, for example:

* **Softmax if multi‑class classification**
* Sigmoid if binary
* Identity (no activation) if regression

---

### Dimension summary

| Element                         | Shape  |
| ------------------------------- | ------ |
| Input $X$                     | (N, 4) |
| Weights $W^{(1)}$             | (3, 4) |
| Bias $b^{(1)}$                | (1, 3) |
| Pre‑activation $Z^{(1)}$      | (N, 3) |
| Hidden layer output $H^{(1)}$ | (N, 3) |
| Weights $W^{(2)}$             | (2, 3) |
| Bias $b^{(2)}$                | (1, 2) |
| Pre‑activation $Z^{(2)}$      | (N, 2) |
| Final output $Y$              | (N, 2) |


---

## **INITIALIZING WHEIGHTS & BIAS**

### **Glorot (Xavier) Initialization**

#### Why the **range and distribution** of the weights matter

To avoid the **vanishing or exploding** of weights and gradients, we initialize weights randomly but with care:

* `np.random.randn()` returns a **standard normal**: mean 0, variance 1.
* If we keep that as‑is and each neuron has many inputs, the weighted sum becomes a **sum of many independent terms**

  $$
  z \;=\; \sum_{i=1}^{n_{\text{in}}} w_i\,x_i
  $$

  * If both $w_i$ and $x_i$ have variance 1, then
    $\operatorname{Var}(z) = n_{\text{in}}\!\cdot\!1\cdot 1 = n_{\text{in}}$.
     With dozens or hundreds of inputs, the **variance of $z$ blows up ⇒ huge activations ⇒ saturated sigmoids/ReLUs ⇒ gradients ≈ 0 (vanishing)**.
  * If we go to the other extreme (weights too small), the signal shrinks ⇒ gradients also tiny.

The fix is to **force a lower variance** on the weights.
Take numbers from $\mathcal N(0,1)$ and **scale by $\sqrt{\text{desired variance}}$**:

$$
w \;=\; \underbrace{\mathcal N(0,1)}_{\text{output of randn}} \times \sqrt{\operatorname{Var}_{\text{desired}}}
$$

---

For a layer with

* $n\_{\text{in}}$ = number of **incoming** units
* $n\_{\text{out}}$ = number of **outgoing** units

set the weight variance to

$$
\operatorname{Var}(W)=\frac{2}{n_{\text{in}}+n_{\text{out}}}.
$$

Thus

$$
w \;=\; \underbrace{\mathcal N(0,1)}_{\text{output of randn}} \times \sqrt{\frac{2}{\,n_{\text{in}}+n_{\text{out}}}}
$$

---

### Common implementations

| Distribution | Sampling rule                                                                                                                               |
| ------------ | ------------------------------------------------------------------------------------------------------------------------------------------- |
| **Normal**   | $W\_{ij}\sim\mathcal{N}!\Bigl(0,;\dfrac{2}{n\_{\text{in}}+n\_{\text{out}}}\Bigr)$                                                         |
| **Uniform**  | $W\_{ij}\sim\mathcal U!\Bigl(,-\sqrt{\dfrac{6}{n\_{\text{in}}+n\_{\text{out}}}},;+\sqrt{\dfrac{6}{n\_{\text{in}}+n\_{\text{out}}}}\Bigr)$ |

> **Python (normal):**
>
> ```python
> W = np.random.randn(n_in, n_out) * np.sqrt(2 / (n_in + n_out))
> ```
>
> **Python (uniform):**
>
> ```python
> limit = np.sqrt(6 / (n_in + n_out))
> W = np.random.uniform(-limit, limit, size=(n_in, n_out))
> ```




In [3]:
# Just for matrix to look good
np.set_printoptions(
    precision=3,      # decimales
    suppress=True,    # evita notación científica
    linewidth=120,    # ancho de línea antes de saltar
    formatter={'float': '{:7.3f}'.format}  # ancho fijo
)

### **1. Hidden Layer**

#### Weights **W_xh**

In [4]:
Wxh = np.random.randn(H_len, entry_len) * np.sqrt(2 / (8 + 3))
print('OG')
print(Wxh)
print('\nTransposed')
print(Wxh.T)

OG
[[  0.501  -0.810   0.148   0.768  -0.089  -0.260   0.606  -0.107]
 [ -0.237  -0.251   0.336   0.056  -0.068   0.129  -0.178   0.166]
 [ -0.103  -0.577  -0.329   0.633   0.367   0.116   0.136  -0.107]]

Transposed
[[  0.501  -0.237  -0.103]
 [ -0.810  -0.251  -0.577]
 [  0.148   0.336  -0.329]
 [  0.768   0.056   0.633]
 [ -0.089  -0.068   0.367]
 [ -0.260   0.129   0.116]
 [  0.606  -0.178   0.136]
 [ -0.107   0.166  -0.107]]


#### Bias **B_h**

In [5]:
Bh = np.zeros(H_len)
Bh

array([  0.000,   0.000,   0.000])

### **2. Output Layer**

#### Weights **W_ho**

In [6]:
Who = np.random.randn(O_len, H_len) * np.sqrt(2 / (3 + 2))

print('OG')
print(Who)
print('\nTransposed')
print(Who.T)

OG
[[  0.394   0.701   0.416]
 [  0.232  -0.486  -0.254]]

Transposed
[[  0.394   0.232]
 [  0.701  -0.486]
 [  0.416  -0.254]]


#### Bias **B_o**

In [7]:
Bo = np.zeros(O_len)
Bo

array([  0.000,   0.000])

---

We’ll develop the **code incrementally**, **sequentially computing the neural network’s forward pass and back‑propagation on a single instance for clear educational visualization.**”

---


In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uciml/pima-indians-diabetes-database")
pima = pd.read_csv(f"{path}/diabetes.csv")
pima.head()

/home/raulserrano/Code/ml_projects/perceptron/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [9]:
Yraw = pima['Outcome'].values
Xraw = pima.drop('Outcome', axis=1).values

In [10]:
def standardize(X):

    mu = X.mean(axis=0)
    std = X.std(axis=0)
    std = np.where(std == 0, 1, std)

    XS = (X - mu) / std

    return XS

In [11]:
X = standardize(Xraw)
Y = np.eye(2)[Yraw]

X_1 = X[0]

print(X)
print()
print(Y)

[[  0.640   0.848   0.150 ...   0.204   0.468   1.426]
 [ -0.845  -1.123  -0.161 ...  -0.684  -0.365  -0.191]
 [  1.234   1.944  -0.264 ...  -1.103   0.604  -0.106]
 ...
 [  0.343   0.003   0.150 ...  -0.735  -0.685  -0.276]
 [ -0.845   0.160  -0.471 ...  -0.240  -0.371   1.171]
 [ -0.845  -0.873   0.046 ...  -0.202  -0.474  -0.871]]

[[  0.000   1.000]
 [  1.000   0.000]
 [  0.000   1.000]
 ...
 [  1.000   0.000]
 [  0.000   1.000]
 [  1.000   0.000]]


## **FORMULAS**

### * **Z**

$$
Z^{(N)} = X \cdot W^{(N)^T} + b^{(N)}
$$

In [12]:
'''
Xi = (1, 8)
Wxh = (3, 8) -->  Wxh.T = (8, 3)
Bh = (1, 3)
'''
#   X_1 @ Wxh = (1,3)

def Z(X, W, B):

    z = (X @ W.T) + B

    return z  

Zh = Z(X_1, Wxh, Bh)

Zh

array([  0.492,  -0.037,  -0.350])

### \* **Sigmoid** **σ(z)**

$$
\sigma(z)=\frac{1}{1+e^{-z}}
$$

**Overflow‑safe equivalent form:**

$$
\sigma(z)=
\begin{cases}
\dfrac{1}{1+e^{-z}}, & z\ge 0,\\[8pt]
\dfrac{e^{\,z}}{1+e^{\,z}}, & z<0.
\end{cases}
$$


* For $z \ge 0$, the factor $e^{-z}$ is small, so it doesn’t blow up.
* For $z < 0$, we use $e^{z}$, which remains tiny (no overflow), and the fraction is algebraically identical to the original sigmoid.


In [13]:
import math as m
import numpy as np


# STABLE SIGMOID FORMULA (Perceptron), **scalar Z**
def sigmoid_esc(Z):
    """
    Numerically stable sigmoid for a single (scalar) input Z.
    Avoids overflow when |Z| is large.
    """
    # f = 1 / (1 + m.exp(-Z))      <-- overflows if Z > ≈700
    # return f

    if Z >= 0:
        return 1 / (1 + m.exp(-Z))
    else:
        ez = m.exp(Z)              # Z is negative → ez is tiny
        return ez / (1 + ez)


# STABLE SIGMOID FORMULA, **array Z**
def sigmoid(Z):
    """
    Numerically stable element‑wise sigmoid for NumPy arrays
    (also works for scalars thanks to broadcasting).
    """

    # # 1) Boolean mask for each element
    # cond = Z >= 0
    #
    # # 2) Two expressions, one for Z >= 0 and another for Z < 0
    # pos = 1 / (1 + np.exp(-Z))          # ‘positive’ part
    # neg = np.exp(Z) / (1 + np.exp(Z))   # ‘negative’ part
    #
    # # 3) Combine both results according to the mask
    # result = np.where(cond, pos, neg)
    #
    # # 4) Return the array with the sigmoid applied
    # return result

    result = np.where(
        Z >= 0,                      # e.g. [False False  True  True  True]
        1 / (1 + np.exp(-Z)),        # Array with the formula for Z >= 0 (A)
        np.exp(Z) / (1 + np.exp(Z))  # Array with the formula for Z < 0  (B)
    )                                # Result = B[i], B[i], A[i], A[i], A[i]

    return result

Yh = sigmoid(Zh)
Yh

array([  0.621,   0.491,   0.413])

### \* **Softmax s(z)**

Converts a vector of logits $\mathbf{z}=(z\_1,;z\_2,\dots,z\_K)$ into a vector of probabilities $\mathbf{p}=(p\_1,;p\_2,\dots,p\_K)$:

$$
\boxed{%
p_k \;=\;
\frac{e^{\,z_k}}
     {\displaystyle\sum_{j=1}^{K} e^{\,z_j}}
}
\quad\text{for } k = 1,\dots,K.
$$

* Each $p_k$ lies in \$(0,1)$ and $\sum\_k p\_k = 1$.
* In practice, one usually subtracts $\max_j z_j$ from every logit before exponentiating to avoid numerical overflow. This does not change the final probabilities, because the common factor cancels out.


In [14]:
import numpy as np


def soft_max(Z):
    """
    Basic soft-max for a 1-D vector (no numerical stabilization).
    """
    ezs = np.exp(Z)

    total = np.sum(ezs)   # sum of exponentials

    result = ezs / total
    return result


# Handles 2-D arrays (batches) as well as 1-D vectors
def softmax_stable(Z, axis=None):
    """
    Numerically stable soft-max.

    Parameters
    ----------
    Z : scalar, 1-D vector, or 2-D array
    axis : int or None, optional
        axis=None  → treat as a 1-D vector (default)
        axis=1     → rows of a 2-D array (batch x classes)
    """
    # 1) Shift by the maximum to avoid overflow in exp
    Z_shift = Z - np.max(Z, axis=axis, keepdims=True)

    # 2) Exponentiate the shifted logits
    e = np.exp(Z_shift)

    # 3) Normalize by the sum along the chosen axis; keep the dimensions
    sum_e = np.sum(e, axis=axis, keepdims=True)
    return e / sum_e


In [15]:
Zo = Z(Yh, Who, Bo)
Zo

Yo = softmax_stable(Zo)
Yo

array([  0.723,   0.277])

### \* **LOSS FUNCTIONS:** 

#### **MSE** 

The **Mean Squared Error (MSE)** for a data set of $n$ observations is

$$
\text{MSE}\;=\;\frac{1}{n}\,\sum_{i=1}^{n}\bigl(\hat{o}_i - y_i\bigr)^2,
$$

where

* $y_i$ is the true (target) value for sample $i$,
* $\hat{o}_i$ is the model’s prediction for that sample,
* $n$ is the total number of samples.

For our network the targets must be in **one‑hot** form:

* **0 → \[1, 0]**
* **1 → \[0, 1]**



In [16]:
def SME(Y, tgt, axis=None):
    """
    Mean Squared Error:
    - axis=None  → scalar with global mean error
    - axis=0     → mean per column (class)  // Not used //
    - axis=1     → mean per row (sample)
    """
    return np.mean((Y - tgt) ** 2, axis=axis)

#### **Cross‑Entropy Loss**

Mean Squared Error **(MSE)** **often becomes problematic and inefficient** for classification:

* With multiple classes ($K>2$) the **gradients** produced by MSE **shrink** as the soft‑max outputs approach the one‑hot targets, so **weight updates slow down**.
* Because **MSE treats every output dimension independently**, it **cannot exploit the *probability‑simplex*** structure that naturally couples the class probabilities.

By contrast, the **soft‑max + cross‑entropy** combination is almost tailor‑made for classification:

* **Clean gradients.**
  After applying soft‑max the derivative of the loss `w.r.t.` the logits is simply

  $$
  \frac{\partial L}{\partial z_k}=p_k - y_k,
  $$

  which stays well‑behaved even when $K$ is large.
* **Faster convergence.**
  The loss penalises confident but wrong predictions much more than uncertain ones, providing a strong corrective signal early in training.
* **Probabilistic meaning.**
  Minimising cross‑entropy is equivalent to maximising the (log) likelihood of the correct class, giving the output a clear interpretation as class probabilities.

---

For a single training example with logits $\mathbf{z}=(z\_1,\dots,z\_K)$ and one‑hot target $\mathbf{y}=(y\_1,\dots,y\_K)$ the categorical cross‑entropy is

$$
\boxed{%
L \;=\; -\sum_{k=1}^{K} y_k \,\log p_k
}\!,
\qquad
p_k=\operatorname{softmax}(z_k)=\frac{e^{z_k}}{\sum_{j=1}^{K} e^{z_j}},
\qquad
y_k\in\{0,1\}.
$$

*If $K=2$ this reduces to the familiar binary‑cross‑entropy / log‑loss formula.*

In [17]:
def cross_entropy(Yo, tgt, axis=None):
    """
    Cross-entropy loss for soft-max outputs.

    Parameters
    ----------
    Yo   : np.ndarray
        Predicted probabilities (output of the soft-max)  
        Shape: (*batch*, n_classes) or any shape compatible with `tgt`.

    tgt  : np.ndarray
        One-hot (or soft) target distribution of the same shape as `Yo`.
        
    axis : int or None, optional
        Dimension along which to sum the class-wise losses.  
        • None  → returns a single scalar (sum over all elements)  
        • int   → returns a loss per slice (e.g. per sample if `axis=1`).

    Returns
    -------
    loss : float or np.ndarray
        Cross-entropy value(s).
    """

    # -------- numerical stability --------------------------------------
    eps = 1e-12                  # small > 0  →  avoids log(0) = −∞
    # np.clip confines Yo to the range [eps, 1]:
    #  • values < eps  are raised to eps
    #  • values between eps and 1 remain unchanged
    #  • the upper bound 1 is redundant (soft-max ≤ 1) but harmless
    safe_p = np.clip(Yo, eps, 1.0)

    # -------- loss ------------------------------------------------------
    # Formula:  L = − Σ  y_k · log(p_k)
    # Multiply element-wise, then sum along the chosen axis
    return -np.sum(tgt * np.log(safe_p), axis=axis)


## **BACKPROPAGATION**

### **Gradient at the Output Layer**

After you combine **the derivative of the cross‑entropy loss with the Jacobian of the soft‑max**, an unexpectedly tidy result pops out:

$$
\boxed{\displaystyle
\frac{\partial L}{\partial z_k}\;=\;\hat{o}_k - y_k
}
$$

Hence, the **vector** you back‑propagate from the output layer is simply

$$
\delta^{(\text{output})}\;=\;\hat{\mathbf{o}}\;-\;\mathbf{y}.
$$

#### Practical advantages

| Advantage               | Why it matters                                         |
| ----------------------- | ------------------------------------------------------ |
| **Very simple formula** | No extra multiplications or cross‑sums are required.   |
| **Stable gradients**    | Avoids saturation even when $p\_k \to 0$ or $1$.   |
| **Faster convergence**  | Provides a strong corrective signal early in training. |


In [18]:
def grad_O(tgt, Yo):

    grad = Yo - tgt

    return grad

gradO = grad_O(Y[0], Yo)
gradO

array([  0.723,  -0.723])

### **Gradient at the Hidden Layer**

#### **Scalar form – neuron by neuron**

$$
\boxed{%
\displaystyle
\delta_i^{(l)}
  \;=\;
\sigma'\!\bigl(z_i^{(l)}\bigr)\,
\sum_{k=1}^{d_{l+1}}
        w_{ik}^{(l+1)}\;\delta_k^{(l+1)}
}\tag{1}
$$

* $z\_i^{(l)}$ — linear input to neuron $i$ in layer $l$
* $\sigma'(z)=a,(1-a)$ — derivative of the sigmoid at that point
* $w\_{ik}^{(l+1)}$ — weight **from neuron $i** to neuron $k$ in layer $l!+!1$
* $\delta\_k^{(l+1)}$ — “blame” (error term) already computed in the next layer

> **Intuition:** “Take the error of each neuron in the layer above, weight it by its connection to me, and finally scale the result by my own sigmoid slope.”

---

#### **Compact matrix form – the whole layer at once**

$$
\boxed{%
\displaystyle
\delta^{(l)}
  = \bigl(W^{(l+1)}\bigr)^{\!\top}\,\delta^{(l+1)}
    \;\odot\;
    a^{(l)}\!\bigl(1-a^{(l)}\bigr)
}\tag{2}
$$

* $W^{(l+1)}!\in!\mathbb{R}^{d\_{l+1}\times d\_{l}}$
  (rows = neurons in layer $l!+!1$, columns = neurons in layer $l$)
* $\delta^{(l)},;a^{(l)}$ — vectors of length $d\_{l}$
* $\odot$ — element‑wise (Hadamard) product

---

### Why prefer the matrix version?

| Aspect                    | Scalar (Eq. 1)                                                                                | Matrix (Eq. 2)                                                                                                                                |
| ------------------------- | --------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------- |
| **Python implementation** | Needs **two loops**: one over neurons $i$ and another over neurons $k$ in the next layer. | A single vectorised line: `delta = W.T @ delta_next * (a * (1 - a))`.                                                                         |
| **Performance**           | Pure‑Python loops are slow and cannot leverage BLAS.                                          | `@` (matrix multiply) dispatches to BLAS/BLIS/OpenBLAS in C, running the block operation far faster.                                          |
| **Scalability**           | Code size and visible operations grow quadratically with layer width.                         | Same code works for any layer size—or even a full **batch** by adding one extra dimension.                                                    |
| **High‑level clarity**    | Good for seeing the neuron‑to‑neuron mechanics.                                               | Encapsulates the **chain rule** for the entire layer: “redistribute the blame” ($W^\top\delta$) then “scale by the slope” ($\sigma'(z)$). |

In practice, Equation (2) is always used; Equation (1) is just the same operation spelled out so you can watch what happens in each individual neuron.

---

#### Sigmoid derivative reminder

$$
\sigma(z)=\frac{1}{1+e^{-z}}, 
\qquad
\sigma'(z)=\sigma(z)\,\bigl(1-\sigma(z)\bigr)=a\,(1-a).
$$


In [19]:
def grad_H(Yh, gradO):

    grad = (Yh*(1-Yh)) * (gradO @ Who)
    return grad

gradH = grad_H(Yh, gradO)
gradH

array([  0.028,   0.215,   0.117])

## **Updating W & B**

In [20]:
print(f'Hidden-layer gradient vector (gradH):\n{gradH}')
print(f'Shape of gradH:\n{gradH.shape}')
print()
print(f'Input vector to the network (Xi):\n{X_1}')
print(f'Shape of Xi:\n{X_1.shape}')

Hidden-layer gradient vector (gradH):
[  0.028   0.215   0.117]
Shape of gradH:
(3,)

Input vector to the network (Xi):
[  0.640   0.848   0.150   0.907  -0.693   0.204   0.468   1.426]
Shape of Xi:
(8,)


In [21]:
def update_weights(Xi, Wxh, Who, Yh, gradH, gradO):

    Wxh = Wxh - (0.1 * np.outer(gradH, Xi))  # Can't use @ tal cual because they are plain
    Who = Who - (0.1 * np.outer(gradO, Yh))  # 1-D vectors, can't transpose them.

    return Wxh, Who



print('OG')
print(Wxh, '\n\n', Who)

Wxh, Who = update_weights(X_1, Wxh, Who, Yh, gradH, gradO)
print()
print('Updated')
print(Wxh, '\n\n', Who)



OG
[[  0.501  -0.810   0.148   0.768  -0.089  -0.260   0.606  -0.107]
 [ -0.237  -0.251   0.336   0.056  -0.068   0.129  -0.178   0.166]
 [ -0.103  -0.577  -0.329   0.633   0.367   0.116   0.136  -0.107]] 

 [[  0.394   0.701   0.416]
 [  0.232  -0.486  -0.254]]

Updated
[[  0.499  -0.813   0.148   0.765  -0.087  -0.260   0.605  -0.111]
 [ -0.251  -0.269   0.333   0.036  -0.053   0.125  -0.188   0.136]
 [ -0.111  -0.587  -0.331   0.622   0.375   0.114   0.130  -0.123]] 

 [[  0.349   0.665   0.386]
 [  0.277  -0.451  -0.224]]


In [22]:
def update_bias(Bh, Bo, gradH, gradO):

    Bh = Bh - 0.1 * gradH
    Bo = Bo - 0.1 * gradO

    return Bh, Bo

print('OG')
print(Bh, '\n\n', Bo)

Bh, Bo = update_bias(Bh, Bo, gradH, gradO)

print()
print('Updated')
print(Bh, '\n\n', Bo)

OG
[  0.000   0.000   0.000] 

 [  0.000   0.000]

Updated
[ -0.003  -0.021  -0.012] 

 [ -0.072   0.072]


# **CLEAN**

In [23]:
class MLP:
   
    def __init__(self, tr_data, tr_target, alpha, epoch):
        
        self.X = self.standardize(tr_data)
        self.target = tr_target
        self.Ycoded = np.eye(2)[self.target]
        self.alpha = alpha

        self.epoch = epoch

        self.n_inputs = self.X.shape[1]

        self.Wxh = np.random.randn(3, self.n_inputs) * np.sqrt(2 / (8 + 3))
        self.Who = np.random.randn(2, 3) * np.sqrt(2 / (3 + 2))

        self.Bh = np.zeros(3)
        self.Bo = np.zeros(2)

        self.loss_rows = []
        # pd.DataFrame(columns=['epoch', 'loss'])

    def standardize(self, X, mode='tr'):

        if mode == 'tr': 

            self.mu = X.mean(axis=0)
            self.std = X.std(axis=0)
            self.std = np.where(self.std == 0, 1, self.std)

            XS = (X - self.mu) / self.std

        elif mode == 'ts':
        
            XS = (X - self.mu) / self.std

        return XS

    def Z(self, Xi, W, B):
        '''
        Xi = (1, N)
        Wxh = (M, N) -->  Wxh.T = (N, M)
        Bh = (1, M)
        '''
        #   X_1 @ Wxh = (1,M)

        z = (Xi @ W.T) + B

        return z 
    

    # STABLE SIGMOID FORMULA, **array Z**
    def sigmoid(self, Z):
        """
        Numerically stable element-wise sigmoid for NumPy arrays
        (also works for scalars thanks to broadcasting).
        """

        # # 1) Boolean mask for each element
        # cond = Z >= 0
        #
        # # 2) Two expressions, one for Z >= 0 and another for Z < 0
        # pos = 1 / (1 + np.exp(-Z))          # ‘positive’ part
        # neg = np.exp(Z) / (1 + np.exp(Z))   # ‘negative’ part
        #
        # # 3) Combine both results according to the mask
        # result = np.where(cond, pos, neg)
        #
        # # 4) Return the array with the sigmoid applied
        # return result

        result = np.where(
            Z >= 0,                      # e.g. [False False  True  True  True]
            1 / (1 + np.exp(-Z)),        # Array with the formula for Z >= 0 (A)
            np.exp(Z) / (1 + np.exp(Z))  # Array with the formula for Z < 0  (B)
        )                                # Result = B[i], B[i], A[i], A[i], A[i]

        return result
    

    # Handles 2-D arrays (batches) as well as 1-D vectors
    def softmax_stable(self, Z, axis=None):
        """
        Numerically stable soft-max.

        Parameters
        ----------
        Z : scalar, 1-D vector, or 2-D array
        axis : int or None, optional
            axis=None  → treat as a 1-D vector (default)
            axis=1     → rows of a 2-D array (batch x classes)
        """
        # 1) Shift by the maximum to avoid overflow in exp
        Z_shift = Z - np.max(Z, axis=axis, keepdims=True)

        # 2) Exponentiate the shifted logits
        e = np.exp(Z_shift)

        # 3) Normalize by the sum along the chosen axis; keep the dimensions
        sum_e = np.sum(e, axis=axis, keepdims=True)
        return e / sum_e


    def cross_entropy(self, Y, tgt, axis=None):
        """
        Cross-entropy loss for soft-max outputs.

        Parameters
        ----------
        Yo   : np.ndarray
            Predicted probabilities (output of the soft-max)  
            Shape: (*batch*, n_classes) or any shape compatible with `tgt`.

        tgt  : np.ndarray
            One-hot (or soft) target distribution of the same shape as `Yo`.
            
        axis : int or None, optional
            Dimension along which to sum the class-wise losses.  
            • None  → returns a single scalar (sum over all elements)  
            • int   → returns a loss per slice (e.g. per sample if `axis=1`).

        Returns
        -------
        loss : float or np.ndarray
            Cross-entropy value(s).
        """

        # -------- numerical stability --------------------------------------
        eps = 1e-12                  # small > 0  →  avoids log(0) = −∞
        # np.clip confines Yo to the range [eps, 1]:
        #  • values < eps  are raised to eps
        #  • values between eps and 1 remain unchanged
        #  • the upper bound 1 is redundant (soft-max ≤ 1) but harmless
        safe_p = np.clip(Y, eps, 1.0)

        # -------- loss ------------------------------------------------------
        # Formula:  L = − Σ  y_k · log(p_k)
        # Multiply element-wise, then sum along the chosen axis
        return -np.sum(tgt * np.log(safe_p), axis=axis)
    
    def grad_O(self, tgt, Y):

        grad = Y - tgt

        return grad
    
    def grad_H(self, Y):

        grad = (Y*(1-Y)) * (self.o_grad @ self.Who)

        return grad

    def update_weights(self, Xi, Yh):

        self.Wxh = self.Wxh - (self.alpha * np.outer(self.h_grad, Xi))  # Can't use @ tal cual because they are plain
        self.Who = self.Who - (self.alpha * np.outer(self.o_grad, Yh))  # 1-D vectors, can't transpose them.

    def update_bias(self):

        self.Bh = self.Bh - self.alpha * self.h_grad
        self.Bo = self.Bo - self.alpha * self.o_grad

    def train(self):

        for epoch in range(self.epoch):
            perm = np.random.permutation(len(self.X))

            for i in perm:
                Xi = self.X[i]
                target_i = self.Ycoded[i]

                Zh = self.Z(Xi, self.Wxh, self.Bh)
                Yh = self.sigmoid(Zh)          # guardar para pesos salida

                Zo = self.Z(Yh, self.Who, self.Bo)
                Yo = self.softmax_stable(Zo)

                loss_i = self.cross_entropy(Yo, target_i)
                self.loss_rows.append((epoch, loss_i))

                self.o_grad = self.grad_O(target_i, Yo)
                self.h_grad = self.grad_H(Yh)

                self.update_weights(Xi, Yh)     # <= pasa Yh
                self.update_bias()

        self.loss_hist = pd.DataFrame(self.loss_rows, columns=['epoch','loss'])
        print('Model trained')

    def get_loss(self):
        return self.loss_hist

    def predict(self, ts_data):

        XS = self.standardize(ts_data, mode='ts')

        self.results = np.zeros(len(XS), dtype=int)

        for i, Xi in enumerate(XS):

            Zh = self.Z(Xi, self.Wxh, self.Bh)
            Yh = self.sigmoid(Zh)

            Zo = self.Z(Yh, self.Who, self.Bo)
            Yo = self.softmax_stable(Zo)

            self.results[i] = int(Yo[1] >= 0.5)

    
    def accuracy(self, act_result):

        comp = self.results == act_result
        accuracy = comp.mean()

        return accuracy


### Function to **split and preprocess data**

In [24]:
def split_process(df, prop, tgt_col):
    # prop = proportion, range(0.0, 1.0)
    # tgt_col = name of the class column

    idx = int(len(df) * prop)
    
    train = df.iloc[0:idx]
    test = df.iloc[idx:]

    tr_tgt = train[tgt_col].values
    train = train.drop(columns=tgt_col)
    train = train.values

    ts_tgt = test[tgt_col].values
    test = test.drop(columns=tgt_col)
    test = test.values

    return train, tr_tgt, test, ts_tgt

train, tr_tgt, test, ts_tgt = split_process(pima, 0.5, 'Outcome')

In [25]:
tr_tgt

array([1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1

In [26]:
model = MLP(train, tr_tgt, 0.1, 20)

model.train()
model.predict(test)
model.accuracy(ts_tgt)

Model trained


np.float64(0.78125)

## **Flexible version** (change Nº neurons in hidden layer)

In [27]:
class MLP2:
   
    def __init__(self, n_hidden, tr_data, tr_target, alpha, epoch):
        
        self.n_hidden = n_hidden
        self.X = self.standardize(tr_data)
        self.target = tr_target
        self.Ycoded = np.eye(2)[self.target]
        self.alpha = alpha

        self.epoch = epoch

        self.n_inputs = self.X.shape[1]

        self.Wxh = np.random.randn(self.n_hidden, self.n_inputs) * np.sqrt(2 / (self.n_inputs + self.n_hidden))
        self.Who = np.random.randn(2, self.n_hidden) * np.sqrt(2 / (self.n_hidden + 2))

        self.Bh = np.zeros(n_hidden)
        self.Bo = np.zeros(2)

        self.loss_rows = []

    def standardize(self, X, mode='tr'):

        if mode == 'tr': 

            self.mu = X.mean(axis=0)
            self.std = X.std(axis=0)
            self.std = np.where(self.std == 0, 1, self.std)

            XS = (X - self.mu) / self.std

        elif mode == 'ts':
        
            XS = (X - self.mu) / self.std

        return XS

    def Z(self, Xi, W, B):
        '''
        Xi = (1, N)
        Wxh = (M, N) -->  Wxh.T = (N, M)
        Bh = (1, M)
        '''
        #   X_1 @ Wxh = (1,M)

        z = (Xi @ W.T) + B

        return z 
    

    # STABLE SIGMOID FORMULA, **array Z**
    def sigmoid(self, Z):
        """
        Numerically stable element-wise sigmoid for NumPy arrays
        (also works for scalars thanks to broadcasting).
        """

        # # 1) Boolean mask for each element
        # cond = Z >= 0
        #
        # # 2) Two expressions, one for Z >= 0 and another for Z < 0
        # pos = 1 / (1 + np.exp(-Z))          # ‘positive’ part
        # neg = np.exp(Z) / (1 + np.exp(Z))   # ‘negative’ part
        #
        # # 3) Combine both results according to the mask
        # result = np.where(cond, pos, neg)
        #
        # # 4) Return the array with the sigmoid applied
        # return result

        result = np.where(
            Z >= 0,                      # e.g. [False False  True  True  True]
            1 / (1 + np.exp(-Z)),        # Array with the formula for Z >= 0 (A)
            np.exp(Z) / (1 + np.exp(Z))  # Array with the formula for Z < 0  (B)
        )                                # Result = B[i], B[i], A[i], A[i], A[i]

        return result
    

    # Handles 2-D arrays (batches) as well as 1-D vectors
    def softmax_stable(self, Z, axis=None):
        """
        Numerically stable soft-max.

        Parameters
        ----------
        Z : scalar, 1-D vector, or 2-D array
        axis : int or None, optional
            axis=None  → treat as a 1-D vector (default)
            axis=1     → rows of a 2-D array (batch x classes)
        """
        # 1) Shift by the maximum to avoid overflow in exp
        Z_shift = Z - np.max(Z, axis=axis, keepdims=True)

        # 2) Exponentiate the shifted logits
        e = np.exp(Z_shift)

        # 3) Normalize by the sum along the chosen axis; keep the dimensions
        sum_e = np.sum(e, axis=axis, keepdims=True)
        return e / sum_e


    def cross_entropy(self, Y, tgt, axis=None):
        """
        Cross-entropy loss for soft-max outputs.

        Parameters
        ----------
        Yo   : np.ndarray
            Predicted probabilities (output of the soft-max)  
            Shape: (*batch*, n_classes) or any shape compatible with `tgt`.

        tgt  : np.ndarray
            One-hot (or soft) target distribution of the same shape as `Yo`.
            
        axis : int or None, optional
            Dimension along which to sum the class-wise losses.  
            • None  → returns a single scalar (sum over all elements)  
            • int   → returns a loss per slice (e.g. per sample if `axis=1`).

        Returns
        -------
        loss : float or np.ndarray
            Cross-entropy value(s).
        """

        # -------- numerical stability --------------------------------------
        eps = 1e-12                  # small > 0  →  avoids log(0) = −∞
        # np.clip confines Yo to the range [eps, 1]:
        #  • values < eps  are raised to eps
        #  • values between eps and 1 remain unchanged
        #  • the upper bound 1 is redundant (soft-max ≤ 1) but harmless
        safe_p = np.clip(Y, eps, 1.0)

        # -------- loss ------------------------------------------------------
        # Formula:  L = − Σ  y_k · log(p_k)
        # Multiply element-wise, then sum along the chosen axis
        return -np.sum(tgt * np.log(safe_p), axis=axis)
    
    def grad_O(self, tgt, Y):

        grad = Y - tgt

        return grad
    
    def grad_H(self, Y):

        grad = (Y*(1-Y)) * (self.o_grad @ self.Who)

        return grad

    def update_weights(self, Xi, Yh):

        self.Wxh = self.Wxh - (self.alpha * np.outer(self.h_grad, Xi))  # Can't use @ tal cual because they are plain
        self.Who = self.Who - (self.alpha * np.outer(self.o_grad, Yh))  # 1-D vectors, can't transpose them.

    def update_bias(self):

        self.Bh = self.Bh - self.alpha * self.h_grad
        self.Bo = self.Bo - self.alpha * self.o_grad

    def train(self):

        for epoch in range(self.epoch):
            perm = np.random.permutation(len(self.X))

            for i in perm:
                Xi = self.X[i]
                target_i = self.Ycoded[i]

                Zh = self.Z(Xi, self.Wxh, self.Bh)
                Yh = self.sigmoid(Zh)          # guardar para pesos salida

                Zo = self.Z(Yh, self.Who, self.Bo)
                Yo = self.softmax_stable(Zo)

                loss_i = self.cross_entropy(Yo, target_i)
                self.loss_rows.append((epoch, loss_i))

                self.o_grad = self.grad_O(target_i, Yo)
                self.h_grad = self.grad_H(Yh)

                self.update_weights(Xi, Yh)     # <= pasa Yh
                self.update_bias()

        self.loss_hist = pd.DataFrame(self.loss_rows, columns=['epoch','loss'])
        print('Model trained')

    def get_loss(self):
        return self.loss_hist

    def predict(self, ts_data):

        XS = self.standardize(ts_data, mode='ts')

        self.results = np.zeros(len(XS), dtype=int)

        for i, Xi in enumerate(XS):

            Zh = self.Z(Xi, self.Wxh, self.Bh)
            Yh = self.sigmoid(Zh)

            Zo = self.Z(Yh, self.Who, self.Bo)
            Yo = self.softmax_stable(Zo)

            self.results[i] = int(Yo[1] >= 0.5)

    
    def accuracy(self, act_result):

        comp = self.results == act_result
        accuracy = comp.mean()

        return accuracy


In [28]:
model = MLP2(16,train, tr_tgt, 0.1, 20)

model.train()
model.predict(test)
model.accuracy(ts_tgt)

Model trained


np.float64(0.8125)

### **Test on other dataset**

In [29]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

data   = load_breast_cancer()
data
X_full = data.data                  # shape (569, 30)
y_full = data.target.astype(int)    # 0 = benign, 1 = malignant
X_tr, X_te, y_tr, y_te = train_test_split(X_full, y_full, test_size=0.4, random_state=42, stratify=y_full)

In [30]:
mlp = MLP2(n_hidden=8, tr_data=X_tr, tr_target=y_tr,
           alpha=0.05, epoch=50)
mlp.train()
mlp.predict(X_te)

print("Test accuracy:", mlp.accuracy(y_te))

Model trained
Test accuracy: 0.9736842105263158


In [31]:
X_full

array([[ 17.990,  10.380, 122.800, ...,   0.265,   0.460,   0.119],
       [ 20.570,  17.770, 132.900, ...,   0.186,   0.275,   0.089],
       [ 19.690,  21.250, 130.000, ...,   0.243,   0.361,   0.088],
       ...,
       [ 16.600,  28.080, 108.300, ...,   0.142,   0.222,   0.078],
       [ 20.600,  29.330, 140.100, ...,   0.265,   0.409,   0.124],
       [  7.760,  24.540,  47.920, ...,   0.000,   0.287,   0.070]], shape=(569, 30))